In [0]:
workingDir = "/mnt/kddcap-data"
outputPathDir = workingDir + "/bronze-nsl-rt.json"   # A subdirectory for our output
checkpointPath = workingDir + "/checkpoint.bronze-nsl-rt.json"      # A subdirectory for our checkpoint & W-A logs
myStreamName = "datamass_stream_2022"                   # An arbitrary name for the stream  

conf = {}
conf["eventhubs.connectionString"] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt("Endpoint=sb://dm2022-training.servicebus.windows.net/;SharedAccessKeyName=ehkeys;SharedAccessKey=IiPSS........=;EntityPath=test1")

conf["eventhubs.consumerGroup"] = "c9"

read_df = (
  spark
    .readStream
    .format("eventhubs")
    .options(**conf)
    .load()
)


streamingQuery = (
    read_df 
    .writeStream 
    .queryName(myStreamName) 
    .trigger(processingTime='3 seconds') 
    .format("json") 
    .option("checkpointLocation", checkpointPath) 
    .start(outputPathDir)
)

# output = read_df.withColumn("body",read_df["body"].cast("string")).select("body")
# display(output)